# Ordinary Least Squares (OLS) regression

### Predictive modelling with machine learning

#### Lecturer: Vegard H. Larsen

### Start by generating some data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# Set the number of regressors or futures
n_features = 5

# Set the number of observations
n_samples = 500

# Set intercept to True/False
has_intercept = True

# Generate some random futures from a Uniform(0,1)
X = np.random.rand(n_samples, n_features)

# Generate the true coefficients, also from a Uniform(0,1)
coeffs = np.random.rand(n_features).round(2)

# The data generating process is y = intercept + sum_i coeff_i * x_i + 0.2*u_i
# where u_i is normally distributed error term,  N(O,1).
if has_intercept:
    intercept = np.random.rand(1).round(2)[0]
    y = intercept + np.dot(X, coeffs) + 0.2*np.random.randn(n_samples)
    y_true = intercept + np.dot(X, coeffs)
else:
    intercept = 0
    y = np.dot(X, coeffs) + 0.2*np.random.randn(n_samples)
    y_true = np.dot(X, coeffs)

In [25]:
print(f'The true coefficients are/is: {coeffs}')
if has_intercept:
    print(f'The true intercept is: {intercept}')

The true coefficients are/is: [0.76 0.25 0.4  0.98 0.05]
The true intercept is: 0.17


In [26]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming n_features, X, y, y_true, intercept, coeffs are already defined
if n_features == 1:
    plt.scatter(X, y, label='Data')
    plt.plot(X, y_true, color='red', label=f'DGP ($y={intercept} + {coeffs[0]}x$)')

    # Randomly select 20 points
    indices = np.random.choice(range(len(X)), 20, replace=False)
    for i in indices:
        # Draw line from point to regression line
        plt.plot([X[i], X[i]], [y[i], y_true[i]], color='green', linestyle='--')

    plt.legend()
    plt.xlabel('$x$-value')
    plt.ylabel('$y$-value')
    plt.show()
else:
    print('Can only plot a two-dimensional figure.')


Can only plot a two-dimensional figure.


The goal now is to recover the data generating process (DGP) by using the observed data, $X$.

### Basic OLS recap:

The ordinary least squares (OLS) method is a linear regression technique that finds the coefficients (also called weights or parameters) that minimize the sum of the squared residuals between the predicted values and the true values. Mathematically, this can be written as the following optimization problem

$$\arg\min_{\beta} \sum_{i=1}^n (y_i - \beta^T x_i)^2$$

where $x_i$ is the $i$-th feature vector, $y_i$ is the $i$-th target, $\beta$ is the coefficient vector (also called the betas). The optimal values of $\beta$ are the OLS coefficients.

To solve this optimization problem, we can set the derivative of the objective function with respect to $\beta$ to zero and solve for $\beta$. This leads to the following closed-form solution for the OLS coefficients:

$$\beta = (X^T X)^{-1} X^T y$$

where $X$ is the feature matrix with shape (n_samples, n_features), $y$ is the target vector with shape (n_samples).

We can easily solve for $\beta$ using NumPy:

In [27]:
if has_intercept == True:
    X_ = np.column_stack((np.ones(n_samples), X))    
    betas = np.linalg.inv(X_.T@X_)@(X_.T@y)
    print(f'The estimated coefficients are: { betas[1:].round(3)}')
    print(f'The estimated intercept is: { betas[0].round(3)}')
else:
    betas = np.linalg.inv(X.T@X)@(X.T@y)
    print(f'The estimated coefficients are: { betas.round(3)}')

The estimated coefficients are: [0.752 0.263 0.424 0.954 0.074]
The estimated intercept is: 0.158


### Let's write a few more lines of code using OOP

We will now use object-oriented programming (OOP) when coding up the ordinary least squares (OLS) method. This is the way every serious package in Python is written, and it is smart to start to think in this framework right away.

In [29]:
class OLS:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.coeffs = None
        self.intercept = None

    def fit(self, X, y):
        # Add intercept term to X if fit_intercept is True
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])

        # Solve the least squares problem
        XTX = np.dot(X.T, X)
        XTY = np.dot(X.T, y)
        self.coeffs = np.linalg.solve(XTX, XTY)

        # Extract intercept term if fit_intercept is True
        if self.fit_intercept:
            self.intercept = self.coeffs[0]
            self.coeffs = self.coeffs[1:]

    def predict(self, X):
        # Add intercept term to X if fit_intercept is True
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
            return np.dot(X, np.hstack([self.intercept, self.coeffs]))
        return np.dot(X, self.coeffs)

In [30]:
# Then we can use the class as follows:

model = OLS(fit_intercept=has_intercept)
model.fit(X, y)
print(f'The estimated coefficients are: {model.coeffs.round(3)}')
print(f'The estimated intercept is: {model.intercept:.3f}')

The estimated coefficients are: [0.752 0.263 0.424 0.954 0.074]
The estimated intercept is: 0.158


In [31]:
model.predict(X)

array([1.50986499, 1.41496878, 1.95560593, 0.70710628, 1.50794276,
       1.79879824, 1.28838213, 1.43491732, 1.1173543 , 0.49509858,
       1.70015474, 0.97217535, 0.84252927, 0.81392321, 1.57375563,
       0.96608714, 2.28524985, 1.77204352, 1.24697238, 1.93295777,
       1.78070943, 1.18260154, 0.66644792, 0.60972765, 1.07514401,
       1.47882826, 1.07709334, 0.7696624 , 1.62803019, 1.57586208,
       1.40754879, 1.04128331, 1.63679766, 1.93942301, 2.11858664,
       1.43130701, 2.25575821, 1.0420427 , 0.98005853, 1.08883435,
       2.17344092, 1.33154168, 1.22443228, 2.20732431, 1.45261415,
       1.00602689, 0.98929538, 0.9117072 , 0.7234552 , 2.18812572,
       1.60126562, 1.42626924, 1.124881  , 0.75171672, 1.29622076,
       1.51023559, 1.12375822, 1.39561499, 2.08740622, 1.22920843,
       1.70415292, 1.55164512, 1.27234046, 1.63448005, 1.68136747,
       0.80949989, 1.59961269, 1.35712603, 1.37061794, 1.15948704,
       2.20993834, 1.7869483 , 1.92759639, 0.65919245, 1.66864

In [32]:
# We can also use the scikit-learn package to estimate the coefficients
import sklearn.linear_model as lm

model_sk = lm.LinearRegression(fit_intercept=has_intercept)
model_sk.fit(X, y)
print(f'The estimated coefficients are: {model_sk.coef_.round(3)}')
print(f'The estimated intercept is: {model_sk.intercept_.round(3)}')

The estimated coefficients are: [0.752 0.263 0.424 0.954 0.074]
The estimated intercept is: 0.158
